### LangChain Output Parsers

In [ ]:
import os

from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

temperature = 0.7
max_tokens = 1500
model_name = "gpt-3.5-turbo-0125"

llm = ChatOpenAI(
    model=model_name,
    temperature=temperature,
    max_tokens=max_tokens,
    openai_api_key=openai_api_key
)

In [7]:
response_schema = [
    ResponseSchema(
        name="bad_string",
        description="This is poorly formatted user response"
    ),
    ResponseSchema(
        name="good_string",
        description="This is well and properly formatted user response"
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

format_instructions = output_parser.get_format_instructions()

print("Format Instructions:")
print(format_instructions)

Format Instructions:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This is poorly formatted user response
	"good_string": string  // This is well and properly formatted user response
}
```


In [ ]:
template = """
    You shall be given a poorly formatted string from a user.
    Reformat it and make sure all the words are spelled correctly.
    
    {format_instructions}
    
    User Input: {user_input}
    
    YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    template=template,
    partial_variables={"format_instructions": format_instructions}
)

prompt_value = prompt.format(
    user_input="""
        Welcome to Londan! I hope u hv a great time her. 
        The wether is nice and the foody is delicious. 
        Don't forgat to visit the Big Ban and the Towar of Londan. Enjoy your stey!
    """
)

In [ ]:
llm_output = llm.invoke(
    prompt_value,
)

print("LLM Output:")
print(llm_output)

In [ ]:
result = output_parser.parse(llm_output.content)
print("Parsed Result:")
print(result)
print("Bad String:", result["bad_string"])
print("Good String:", result["good_string"])